## Check List 4 - PlanningPipeline
*\* ~~Strikethrough~~ means no-error but none-functional cases.*
* **4.1 PlanningScene**  
  - create_binder
    - actor: PlacePlane, Gripper2Tool, SweepTool
    - handle: Grasp2Point, PlacePoint, SweepPoint
  - create_subject
    - subject: CustomObject, SweepLineTask
  - initialize_state
* **4.2 MotionPlanner**  
  - MoveitPlanner
    - pick, place, sweep
  - EtaslPlanner
    - pick, place, ~~sweep~~
* **4.3 TaskPlanner**  
  - RRT
    - pick & place & sweep
      - single process
      - multi process

## set running directory to project source

In [1]:
import os
import numpy as np
import time
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 4.1 PlanningScene

##### initialize CombinedRobot and GeometryScene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *
from pkg.geometry.builder.scene_builder import SceneBuilder

s_builder = SceneBuilder(None)   # create scene builder without detector for virtual scene
crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)), None)]
                     , connection_list=[False])
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)
gscene.show_pose(crob.home_pose)

connection command:
indy0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


##### add geometries

In [3]:
from pkg.geometry.geometry import *

# add environments (fixed=True for non-movable geometries)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=np.random.rand(3)*0, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
#                            rpy=np.random.rand(3)*np.pi/16, color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.5,-0.2,0),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0.5,0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="floor")
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,0), 
                          rpy=(0,0,0), color=(0.2,0.2,0.8,1), display=True, fixed=True, collision=False)
 
# add movable (fixed=False for movable geometries)
box1 = gscene.create_safe(GEOTYPE.BOX, "box1", "base_link", (0.05,0.05,0.05), (0.3,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.2,0.2,1), display=True, fixed=False, collision=True)

obstacle = gscene.create_safe(GEOTYPE.BOX, "obstacle", "base_link", (0.05,0.05,0.05), (0.5,0.4,0.031), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=False, collision=True)

##### create PlanningScene

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

##### create_binder
- Binders (or Actors) are binding points where objects can be attached (or binded)
- Examples are 
  - PlacePlane: plane for object placement
  - Gripper2Tool: 2-finger gripper tool for grasp objects
  - SweepTool: action point to pass waypoints for sweep task

In [5]:
from pkg.planning.constraint.constraint_actor import PlacePlane, Gripper2Tool, SweepTool

#### create PlacePlane on geometry "floor" and "goal"
* when **point** is set to *None*, the entire upper surface of the geometry becomes valid binding area.
* when **point** is set, the specific point becomes the only valid binding point.

In [6]:
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=(0,0,0.005))

#### add collision boundary for gripper base

In [7]:
# - set link_name="indy0_tcp" to attach the geometry to end-effector link
# - it can be labeled as fixed=True, as it is "fixed" on the indy0_tcp link
gripper =  gscene.create_safe(GEOTYPE.BOX, "gripper", link_name="indy0_tcp", 
                                dims=(0.02,0.1,0.1), center=(0,0,0.05), rpy=(0,0,0), 
                                color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True)

# add gripper fingers - By setting parent="gripper", the position of geometry can be set relative to the parent geometry
finger1 =  gscene.create_safe(GEOTYPE.BOX, "finger1", link_name="indy0_tcp",
                              dims=(0.04,0.02,0.1), center=(0,0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")
finger2 =  gscene.create_safe(GEOTYPE.BOX, "finger2", link_name="indy0_tcp", 
                              dims=(0.04,0.02,0.1), center=(0,-0.04,0.07), rpy=(0,0,0), 
                              color=(0.8,0.2,0.2,0.5), display=True, fixed=True, collision=True, parent="gripper")

# create Gripper2Tool binder
# Gripper2Tool is a 2-finger gripper, which can rotate along z-direction.
# To align the z-direction with the 2 fingers, rotate by 90 degree along roll axis.
# The gripping point is (0,0,0.11) in local coordinate of "gripper" geometry
pscene.create_binder(bname="grip0", gname="gripper", _type=Gripper2Tool, point=(0,0,0.11), rpy=(-np.pi/2,0,0))

#### Add virtual (no-collision) sweep face. the point is 0.2 m away from the "indy0_tcp" link

In [8]:
# To match the z-direction with the target surface, the geometry is rotated 180 degrees in pitch-axis.
sweep_face =  gscene.create_safe(GEOTYPE.BOX, "sweep_face", link_name="indy0_tcp", 
                                dims=(0.05,0.05,0.001), center=(0,0,0.2), rpy=(0,np.pi,0), 
                                color=(0.2,0.2,0.8,0.1), display=True, fixed=True, collision=False)
                                 
# create SweepTool binder
pscene.create_binder(bname="sweep_face", gname="sweep_face", _type=SweepTool, point=(0,0,0), rpy=(0,0,0))

##### create_subject
* Subject describes the tasks in the planning scene.
* There are 2 categories in subject:
  1. Object: The object has grip points and placement points for pick&place task
  2. Task: The task is can be any non-physical task. Check SweepLineTask for example
* The subjects can be composed of multiple action points. Examples are:
  1. Grasp2Point: grasping point for 2-finger gripper. 
  2. PlacePoint: The point to place object.
  3. SweepPoint: A waypoint for SweepLineTask.
  * The above 3 action points inherit DirectePoint, for which the orientation is free along z-axis. 
  * If "point" parameter is not set, the entire upper surface is becomes valid action area.

#### create box object with grasping points along positive & negative y-direction and placement point in the bottom face

In [9]:
from pkg.planning.constraint.constraint_subject import BoxObject
box_obj = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject)

#### create sweep task with 2 waypoints

In [10]:
from pkg.planning.constraint.constraint_subject import SweepLineTask, SweepPoint
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])}
                             )
sweep.fix_direction = False

##### initialize_state
* initialize_state(robot_pose) updates robot pose and gets corresponding binding status of current scene.
* state.node of ('floor', 0) means the first subject (object) is placed on the floor and the second subject (sweep) has passed 0 waypoints

In [11]:
initial_state = pscene.initialize_state(crob.home_pose)
print(initial_state.node)

('floor', 0)


## 4.2 MotionPlanner

In [ ]:
# motion test script
def motion_fn(from_state, to_node=None, to_state=None, try_count=10, **kwargs):
    assert to_node is not None or to_state is not None, "Either to_node or to_state should be specified"
    for _ in range(try_count):
        if to_node is not None:
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node)
            to_state = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)

        pscene.set_object_state(from_state)
        gscene.show_pose(from_state.Q)
        subjects, sub_ok = pscene.get_changing_subjects(from_state, to_state)
        if len(subjects) == 1:
            show_action_point(pscene, 
                              btf=to_state.binding_state[subjects[0]], 
                              Q=from_state.Q)
        with gtimer.block("plan"):
            Traj, LastQ, error, success, binding_list = mplan.plan_transition(from_state, to_state, **kwargs)
        if success:
            break
    if success:
        pscene.set_object_state(from_state)
        gscene.show_motion(Traj, period=0.01)
        end_state = pscene.rebind_all(binding_list, LastQ)
        gscene.clear_highlight()
    else:
        print("Solution not found. Please try again, try to find error if it keeps failing.")
        raise(RuntimeError("Motion plan failure"))
    return end_state

##### MoveitPlanner
* *get_available_binding_dict()* gets available binding states for each subject in a dictionary
* *sample_leaf_state samples()* target state with given available_binding_dict and target node
* *rebind_all()* updates binding state and returns the resultant state
* The motions tested in this section are:
  - pick: move the object to "gripper"
  - place: move the object to "goal"
  - sweep: 
    1) approach to waypoint 1
    2) sweep to waypoint 2
    3) return to home pose

In [13]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.utils.code_scraps import show_action_point
mplan = MoveitPlanner(pscene)
mp_kwargs = {}

gtimer = GlobalTimer.instance()
gtimer.reset()

#### Pick and place

In [14]:
pick_state = motion_fn(initial_state, to_node=("gripper", 0), **mp_kwargs)

In [15]:
place_state = motion_fn(pick_state, to_node=("goal", 0), **mp_kwargs)

#### Sweep

In [16]:
sweep_approached = motion_fn(place_state, to_node=("goal", 1), **mp_kwargs)

In [17]:
sweep_finished = motion_fn(sweep_approached, to_node=("goal", 2), **mp_kwargs)

#### homing (joint motion)

In [18]:
to_state = sweep_finished.copy(pscene)
to_state.Q = initial_state.Q
home_returned = motion_fn(sweep_finished, to_state=to_state, **mp_kwargs)

##### EtaslPlanner
* same usage as the MoveitPlanner
  - pick, place, ~~sweep~~
* eTaSl is not planner, but Model Predictive Controller-based trajectory generator. Thus, it shows poor obstacle avoidance and fails manytimes.
* if it fails, just try the whole **Pick and place** sequence multiple times

In [19]:
from pkg.planning.motion.etasl.etasl_planner import EtaslPlanner
from pkg.utils.code_scraps import show_action_point
mplan = EtaslPlanner(pscene)
mp_kwargs = dict(timeout=1, vel_conv=0, err_conv=1e-3)

gtimer = GlobalTimer.instance()
gtimer.reset()

#### Pick and place

In [20]:
pick_state = motion_fn(initial_state, to_node=("gripper", 0), try_count=50, **mp_kwargs)

In [21]:
place_state = motion_fn(pick_state, to_node=("goal", 0), try_count=100, **mp_kwargs)

eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution
eTaSL exception: optimization failed during execution


#### homing (joint motion)

In [22]:
to_state = place_state.copy(pscene)
to_state.Q = initial_state.Q
home_returned = motion_fn(place_state, to_state=to_state, **mp_kwargs)

===================== plan joint manipulation =====================


## 4.3 TaskPlanner

##### create PlanningPipeline

In [12]:
from pkg.planning.pipeline import PlanningPipeline
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
ppline = PlanningPipeline(pscene)
mplan = MoveitPlanner(pscene)
ppline.set_motion_planner(mplan)

In [13]:
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan.motion_filters = [GraspChecker(pscene)]

In [14]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


##### RRT

In [15]:
from pkg.planning.task.rrt import TaskRRT
ppline.set_task_planner(TaskRRT(pscene))

- pick & place & sweep
  - single process
  - set display=True to check all intermediate plan segments

In [17]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True,
              timeout_loop=100, multiprocess=False, timeout=1, timeout_constrained=5,
              display=True)
schedules = ppline.tplan.find_schedules(at_home=True)
if len(schedules)==0:
    schedules = ppline.tplan.find_schedules(at_home=False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (1.02/100.0 s, steps/err: 29(1015.03205299 ms)/0.00170291663591)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->2 (2.51/100.0 s, steps/err: 40(1491.574049 ms)/0.000693529102707)
try: 0 - ('floor', 0)->('gripper', 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('gripper', 0) = success
branching: 0->3 (3.69/100.0 s, steps/err: 36(1170.09210587 ms)/0.00155243933796)
try: 1 - ('floor', 1)->('floor', 2)
try constrained motion
joint max
constrained motion tried: False
Motion Plan Failure
result: 1 - ('floor', 1)->('floor', 2) = fail
try: 2 - ('gripper', 0)->('floor', 0)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('gripper', 0)->('floor', 0) = fail
try: 0 - ('fl

- pick & place & sweep  
  - multi process

In [22]:
ppline.search(initial_state, goal_nodes=[("goal", 2)], verbose=True, 
              timeout_loop=100, multiprocess=True, timeout=1, timeout_constrained=5)
schedules = ppline.tplan.find_schedules(at_home=True)
if len(schedules)==0:
    schedules = ppline.tplan.find_schedules(at_home=False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
ppline.play_schedule(ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0]), period=0.01)

Use 36/36 agents
try: 0 - ('floor', 0)->('floor', 1)
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
transition motion tried: True
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
try transition motion
try: 0 - ('floor', 0)->('gripper', 0)
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
result: 0 - ('floor', 0)->('floor', 1) = success
branching: 0->1 (0.13/100.0 s, steps/err: 24(60.0700378418 ms)/0.00170013638382)
branching: 0->2 (0.14/100.0 s, steps/err: 16(68.5648918152 ms)/0.0013614137646)
Motion Filer Failure: GraspChecker
transition motion tried: True
transition motion tried: True
try: 2 - ('floor', 1)->('floor', 2)
transition motion tried: True
try constrained motion
transition motion tried: True
result: 0 - ('floor', 0)->('floor', 1) = success
result: 0 - ('f

branching: 0->15 (0.67/100.0 s, steps/err: 15(83.3458900452 ms)/0.00136225321108)
try: 19 - ('floor', 2)->('gripper', 2)
try transition motion
try transition motion
try: 18 - ('gripper', 2)->('goal', 2)
result: 9 - ('floor', 2)->('gripper', 2) = success
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
result: 0 - ('floor', 0)->('gripper', 0) = success
result: 12 - ('gripper', 0)->('goal', 0) = fail
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 12 - ('gripper', 0)->('floor', 0) = fail
try: 0 - ('floor', 0)->('gripper', 0)
branching: 2->17 (0.69/100.0 s, steps/err: 41(331.054210663 ms)/1.05211478116)
result: 18 - ('gripper', 2)->('goal', 2) = fail
try: 16 - ('floor', 1)->('floor', 2)
try: 12 - ('gripper', 0)->('gripper', 1)
try: 20 - ('gripper', 0)->('gripper', 1)
Motion Filer Failure: GraspChecker
try: 18 - ('gripper', 2)->('floor', 2)
branching: 9->18 (0.71/100.0 s, steps/err: 42(156.679868698 ms)/0.00144651769957)
try constrained m

try: 25 - ('gripper', 1)->('gripper', 2)
end
constrained motion tried: True
try transition motion
transition motion tried: True
end
result: 0 - ('floor', 0)->('gripper', 0) = fail
try constrained motion
branching: 20->31 (0.98/100.0 s, steps/err: 66(229.619026184 ms)/0.00147824316283)
constrained motion tried: True
try: 8 - ('floor', 2)->('gripper', 2)
branching: 0->30 (0.98/100.0 s, steps/err: 19(182.332038879 ms)/0.000855958246785)
try: 25 - ('gripper', 1)->('gripper', 2)
try constrained motion
transition motion tried: True
try: 12 - ('gripper', 0)->('gripper', 1)
try transition motion
Motion Filer Failure: GraspChecker
try transition motion
try: 10 - ('floor', 2)->('gripper', 2)
result: 12 - ('gripper', 0)->('gripper', 1) = success
try constrained motion
result: 2 - ('floor', 1)->('floor', 2) = success
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
result: 20 - ('gripper', 0)->('gripper', 1) = success
transition motion tried: True
result: 8 - ('floor', 2)->('

try transition motion
try constrained motion
try: 19 - ('floor', 2)->('gripper', 2)
branching: 0->48 (1.3/100.0 s, steps/err: 12(125.09894371 ms)/0.00160506358998)
constrained motion tried: True
branching: 26->47 (1.3/100.0 s, steps/err: 67(165.621995926 ms)/0.00119282578258)
transition motion tried: True
transition motion tried: True
try transition motion
try: 0 - ('floor', 0)->('floor', 1)
try transition motion
Motion Filer Failure: GraspChecker
Motion Filer Failure: GraspChecker
result: 0 - ('floor', 0)->('floor', 1) = success
result: 6 - ('floor', 1)->('floor', 2) = success
result: 19 - ('floor', 2)->('gripper', 2) = fail
result: 12 - ('gripper', 0)->('gripper', 1) = success
result: 45 - ('gripper', 2)->('goal', 2) = fail
end
branching: 6->50 (1.34/100.0 s, steps/err: 41(536.931037903 ms)/2.66826149434)
branching: 0->49 (1.34/100.0 s, steps/err: 15(89.5898342133 ms)/0.00151340466566)
try: 0 - ('floor', 0)->('gripper', 0)
try: 40 - ('gripper', 2)->('floor', 2)
transition motion trie

try: 64 - ('gripper', 2)->('goal', 2)
transition motion tried: True
Motion Filer Failure: GraspChecker
try constrained motion
try transition motion
result: 12 - ('gripper', 0)->('gripper', 1) = success
constrained motion tried: True
try: 26 - ('gripper', 0)->('gripper', 1)
branching: 0->65 (1.51/100.0 s, steps/err: 22(131.905078888 ms)/0.00147621096785)
result: 42 - ('goal', 0)->('goal', 1) = success
transition motion tried: True
Motion Filer Failure: GraspChecker
result: 24 - ('gripper', 1)->('gripper', 2) = success
result: 0 - ('floor', 0)->('floor', 1) = success
try transition motion
result: 20 - ('gripper', 0)->('floor', 0) = fail
branching: 0->69 (1.53/100.0 s, steps/err: 16(49.8778820038 ms)/0.00100082674066)
branching: 12->66 (1.52/100.0 s, steps/err: 23(111.993074417 ms)/0.00145482751647)
branching: 24->68 (1.53/100.0 s, steps/err: 41(566.205978394 ms)/2.68620336134)
branching: 25->64 (1.52/100.0 s, steps/err: 41(491.938829422 ms)/2.4749425379)
result: 64 - ('gripper', 2)->('go

result: 26 - ('gripper', 0)->('gripper', 1) = success
branching: 20->83 (1.74/100.0 s, steps/err: 46(148.970842361 ms)/0.00100610130046)
Motion Plan Failure
Motion Filer Failure: GraspChecker
end
++ adding return motion to acquired answer ++
try: 82 - ('goal', 1)->('goal', 2)
branching: 26->84 (1.75/100.0 s, steps/err: 56(221.58908844 ms)/0.00144354581924)
result: 67 - ('goal', 1)->('goal', 2) = fail
try transition motion
result: 12 - ('gripper', 0)->('goal', 0) = fail
result: 42 - ('goal', 0)->('goal', 1) = success
branching: 42->85 (1.76/100.0 s, steps/err: 70(285.920143127 ms)/0.001358655788)
try: 33 - ('gripper', 1)->('gripper', 2)
try: 85 - ('goal', 1)->('goal', 2)
try: 80 - ('goal', 0)->('goal', 1)
try constrained motion
try transition motion
constrained motion tried: True
try constrained motion
transition motion tried: True
try constrained motion
try: 80 - ('goal', 0)->('gripper', 0)
result: 41 - ('gripper', 1)->('gripper', 2) = success
try: 42 - ('goal', 0)->('gripper', 0)
Moti

result: 20 - ('gripper', 0)->('goal', 0) = fail
result: 44 - ('gripper', 0)->('goal', 0) = fail
Goal reached
constrained motion tried: True
branching: 28->101 (2.06/100.0 s, steps/err: 41(425.333023071 ms)/2.34204851457)
try transition motion
result: 99 - ('gripper', 2)->('goal', 2) = fail
try: 99 - ('gripper', 2)->('floor', 2)
branching: 77->103 (2.07/100.0 s, steps/err: 41(350.374937057 ms)/2.54549203742)
result: 78 - ('goal', 1)->('goal', 2) = success
try transition motion
result: 33 - ('gripper', 1)->('gripper', 2) = success
try: 105 - ('gripper', 2)->('goal', 2)
end
end
++ adding return motion to acquired answer ++
++ adding return motion to acquired answer ++
branching: 33->105 (2.09/100.0 s, steps/err: 41(325.837135315 ms)/2.60004671691)
try: 12 - ('gripper', 0)->('gripper', 1)
constrained motion tried: True
branching: 78->104 (2.1/100.0 s, steps/err: 41(377.447843552 ms)/0.230592349091)
try transition motion
joint max
try: 44 - ('gripper', 0)->('goal', 0)
try: 12 - ('gripper', 

result: 98 - ('goal', 1)->('goal', 2) = fail
transition motion tried: True
end
Goal reached
branching: 87->115 (2.23/100.0 s, steps/err: 41(344.138145447 ms)/1.56439911522)
======================= terminated 2: first answer acquired from other agent ===============================
result: 12 - ('gripper', 0)->('gripper', 1) = success
======================= terminated 27: first answer acquired from other agent ===============================
branching: 12->118 (2.25/100.0 s, steps/err: 29(130.825042725 ms)/0.00157046699949)
constrained motion tried: True
======================= terminated 10: first answer acquired from other agent ===============================
result: 42 - ('goal', 0)->('gripper', 0) = success
end
constrained motion tried: True
result: 83 - ('gripper', 1)->('gripper', 2) = success
branching: 42->120 (2.25/100.0 s, steps/err: 110(709.125995636 ms)/0.00122029534311)
++ adding return motion to acquired answer ++
end
======================= terminated 0: first answer acq

branching: 22->132 (2.31/100.0 s, steps/err: 41(362.020969391 ms)/0.617042844032)
======================= terminated 35: first answer acquired from other agent ===============================
======================= terminated 30: first answer acquired from other agent ===============================
Goal reached
Goal reached
end
constrained motion tried: True
Goal reached
result: 92 - ('goal', 1)->('goal', 2) = success
branching: 92->135 (2.36/100.0 s, steps/err: 41(355.496883392 ms)/0.371561428319)
++ adding return motion to acquired answer ++
transition motion tried: False
Motion Plan Failure
result: 12 - ('gripper', 0)->('floor', 0) = fail
======================= terminated 6: first answer acquired from other agent ===============================
======================= terminated 34: first answer acquired ===============================
======================= terminated 19: first answer acquired ===============================
Goal reached
transition motion tried: False
Motion Pl